<img src="./assets/cover.png" alt="Week 5: TensorFlow & Keras">

-----------------------------

<img src="https://www.gstatic.com/devrel-devsite/prod/v2484c9574f819dcf3d7ffae39fb3001f4498b2ece38cec22517931d550e19e7d/tensorflow/images/lockup.svg" alt="TensorFlow" width='200' align='left'>

# TensorFlow Basics

TensorFlow is a free and open-source end-to-end platform for machine learning and artificial intelligence which gives developers the ability to easily build and deploy ML-powered applications. It was developed by the Google Brain team for internal Google use in research and production, then brought to public in 2015 and updated to TensorFlow 2.0 in 2019.

TensorFlow can be used in a wide variety of programming languages, including Python, C++, Javascript, Java, and could run on various platforms like desktop, server, web front-end, and embedded systems, etc.

It supports the following:

* Multidimensional-array based numeric computation (similar to <a href="https://numpy.org/" class="external">NumPy</a>.)
* GPU and distributed processing
* Automatic differentiation
* Model construction, training, and export
* And more

In [1]:
import tensorflow as tf

tf.__version__

'2.8.0'

When [properly configured](https://www.tensorflow.org/install/gpu), TensorFlow can use accelerator hardware like GPUs to execute operations very quickly.

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow **IS** using the GPU")
else:
    print("TensorFlow **IS NOT** using the GPU")

TensorFlow **IS** using the GPU


## Tensors

TensorFlow operates on multidimensional arrays or _tensors_ represented as [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) objects. Here is a two-dimensional tensor:

In [3]:
x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])
x

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
x.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

The most important attributes of a `tf.Tensor` are its `shape` and `dtype`:

* `Tensor.shape`: tells you the size of the tensor along each of its axes.
* `Tensor.dtype`: tells you the type of all the elements in the tensor.

TensorFlow implements standard mathematical operations on tensors, as well as many operations specialized for machine learning.

For example:

In [5]:
x + x

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [6]:
5 * x

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 5., 10., 15.],
       [20., 25., 30.]], dtype=float32)>

In [7]:
x @ tf.transpose(x)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [8]:
tf.concat([x, x, x], axis=0)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [1., 2., 3.],
       [4., 5., 6.],
       [1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [9]:
tf.reduce_sum(x)

<tf.Tensor: shape=(), dtype=float32, numpy=21.0>

In [10]:
tf.nn.softmax(x, axis=1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.09003057, 0.24472848, 0.6652409 ],
       [0.09003057, 0.24472848, 0.6652409 ]], dtype=float32)>

## Variables

Normal `tf.Tensor` objects are immutable. To store model weights (or other mutable state) in TensorFlow use a [`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable).

In [11]:
var = tf.Variable([0.0, 0.0, 0.0])
var

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [12]:
var.assign([1, 2, 3])

<tf.Variable 'UnreadVariable' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [13]:
var[0].assign(4)

<tf.Variable 'UnreadVariable' shape=(3,) dtype=float32, numpy=array([4., 2., 3.], dtype=float32)>

In [14]:
var.assign_add([1, 1, 1])

<tf.Variable 'UnreadVariable' shape=(3,) dtype=float32, numpy=array([5., 3., 4.], dtype=float32)>

## Automatic differentiation

<a href="https://en.wikipedia.org/wiki/Gradient_descent" class="external">_Gradient descent_</a> and related algorithms are a cornerstone of modern machine learning.

To enable this, TensorFlow implements automatic differentiation (autodiff), which uses calculus to compute gradients. Typically you'll use this to calculate the gradient of a model's _error_ or _loss_ with respect to its weights.

### Computing gradients

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the *forward* pass.  Then, during the *backward pass*, TensorFlow traverses this list of operations in reverse order to compute gradients.

Here is a simple example:

In [15]:
x = tf.Variable(1.0)
y = tf.Variable(2.0, trainable=False)

def f(x, y):
    return x**2 + 2*x - 5 - y

f(x, y)

<tf.Tensor: shape=(), dtype=float32, numpy=-4.0>

At `x = 1.0`, `y = f(x) = (1**2 + 2*1 - 5) = -2`.

The derivative of `y` is `y' = f'(x) = (2*x + 2) = 4`. TensorFlow can calculate this automatically:

### Gradient tapes

TensorFlow provides the `tf.GradientTape` API for automatic differentiation; that is, computing the gradient of a computation with respect to some inputs, usually `tf.Variable`s.
TensorFlow "records" relevant operations executed inside the context of a `tf.GradientTape` onto a "tape". TensorFlow then uses that tape to compute the gradients of a "recorded" computation using [reverse mode differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

In [16]:
with tf.GradientTape() as tape:
    res = f(x, y)

g_x = tape.gradient(res, [x, y])
g_x

[<tf.Tensor: shape=(), dtype=float32, numpy=4.0>, None]

This simplified example only takes the derivative with respect to a single scalar (`x`), but TensorFlow can compute the gradient with respect to any number of non-scalar tensors simultaneously. (already updated code)

## Eager Execution and Graphs

For all codes we executed above are under a running mode of TensorFlow called *Eager Execution*. It evaluates values instantly rather than building a computation graph for later use which is compulsory in TensorFlow 1.0. Eager Execution is exactly like how Python scripts does.

While we can use TensorFlow interactively like any Python library, TensorFlow also provides tools for:

* **Performance optimization**: to speed up training and inference.
* **Export**: so you can save your model when it's done training.

These require that you use `tf.function` to separate your pure-TensorFlow code from Python.

In [17]:
@tf.function
def my_func(x):
    print('Tracing...')
    y = x + 5
    return tf.reduce_sum(y)

The first time you run the `tf.function`, although it executes in Python, it captures a complete, optimized graph representing the TensorFlow computations done within the function.

In [18]:
x = tf.constant([1, 2, 3])
my_func(x)

Tracing...


<tf.Tensor: shape=(), dtype=int32, numpy=21>

On subsequent calls TensorFlow only executes the optimized graph, skipping any non-TensorFlow steps. Below, note that `my_func` doesn't print _tracing_ since `print` is a Python function, not a TensorFlow function.

In [19]:
x = tf.constant([10, 9, 8])
my_func(x)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

A graph may not be reusable for inputs with a different _signature_ (`shape` and `dtype`), so a new graph is generated instead:

In [20]:
x = tf.constant([10.0, 9.1, 8.2], dtype=tf.float32)
my_func(x)

Tracing...


<tf.Tensor: shape=(), dtype=float32, numpy=42.3>

These captured graphs provide two benefits:

* In many cases they provide a significant speedup in execution.
* You can export these graphs, using `tf.saved_model`, to run on other systems like a [server](https://www.tensorflow.org/tfx/serving/docker) or a [mobile device](https://www.tensorflow.org/lite/guide), no Python installation required.

In short, graphs are extremely useful and let your TensorFlow run fast, run in parallel, and run efficiently on multiple devices.

However, you still want to define your machine learning models (or other computations) in Python for convenience, and then automatically construct graphs when you need them. And note that the process of creating graphs has some overhead.

## Modules, layers, and models

`tf.Module` is a class for managing your `tf.Variable` objects, and the `tf.function` objects that operate on them. One of significant features of Module is that we can save and load a Module anytime from or to multiple platforms. We will see this in Keras example.

In [21]:
class MyModule(tf.Module):
    def __init__(self, value):
        self.weight = tf.Variable(value)

    @tf.function
    def multiply(self, x):
        return x * self.weight

In [22]:
mod = MyModule(3)
mod.multiply(tf.constant([1, 2, 3]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([3, 6, 9])>

The `tf.keras.layers.Layer` and `tf.keras.Model` classes build on `tf.Module` providing additional functionality and convenience methods for building, training, and saving models.

A model is, abstractly:

+ A function that computes something on tensors (a forward pass)
+ Some variables that can be updated in response to training

## Next —— tf.keras

You may have noticed that classes `Layer` and `Model` are both provided by package `tf.keras` which is the next important role for this session. In the past, Keras is an independent package based on TF, but now it has become an indispensable part of TensorFlow 2.0 ecosystem.

Keras is built upon TensorFlow and provides us high-level encapsulation on data preprocessing, building model, training, and so on. It significantly simplifies workflow of modern data scientists and enables us quickly transform data science ideas into practice.